# OTX API DEMO (Anomalib Example)

## Customization Training API

Select a framework & import adapter modules.

We'll choose Anomalib here, and we'll import the following modules.

## Prepare Dataset & DataLoader
1. Prepare a dataset and enter path into Dataset

    - Convert to OTX's DatasetEntity and Label Schema by leveraging Datumaro's features through paths (path -> Datumaro -> OTX DatasetEntity & LabelSchema)

In [1]:
from otx.v2.adapters.torch.lightning.anomalib import Dataset
dataset = Dataset(
    train_data_roots="../../../../tests/assets/anomaly/hazelnut/train",
    val_data_roots="../../../../tests/assets/anomaly/hazelnut/test",
    test_data_roots="../../../../tests/assets/anomaly/hazelnut/test",
)

To use wandb logger install it using `pip install wandb`


/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataloader = dataset.train_dataloader(batch_size=32)
print(f"Dataset type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")

/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/datumaro/__init__.py:7: DeprecationWarning: We are planning to clean up the entities in datumaro/__init__.py until datumaro==1.5.0. This will affect the following import pattern: "import datumaro as dm; dm.<entity>". If you are using this pattern in your code, please revisit it after upgrading datumaro>=1.5.0.
  warnings.warn(


Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 1
Dataset size: 28


In [3]:
val_dataloader = dataset.val_dataloader(batch_size=32)
print(f"Dataset type: {type(val_dataloader)}")
print(f"Length of DataLoader: {len(val_dataloader)}")
print(f"Dataset size: {len(val_dataloader.dataset)}")

Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 1
Dataset size: 23


In [4]:
test_dataloader = dataset.test_dataloader()
print(f"Dataset type: {type(test_dataloader)}")
print(f"Length of DataLoader: {len(test_dataloader)}")
print(f"Dataset size: {len(test_dataloader.dataset)}")

Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 23
Dataset size: 23


## Prepare Model
Config to build the model. -> Provide function for building models so that each framework's config can be used

    - Users can build a torch.nn.Module via config as well

In [5]:
from otx.v2.adapters.torch.lightning.anomalib import get_model
model_config = {
    "model": {
        "name": "padim",
        "backbone": "resnet18",
        "pre_trained": True,
        "layers": ["layer1", "layer2", "layer3"],
        "normalization_method": "min_max",
        "input_size": [256, 256],
    }
}
model = get_model(model_config)
print(f"Model type: {type(model)}")

/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: DeprecationWarning: From v0.10 an `'Binary*'`, `'Multiclass*', `'Multilabel*'` version now exist of each classification metric. Moving forward we recommend using these versions. This base metric will still work as it did prior to v0.10 until v0.11. From v0.11 the `task` argument introduced in this metric will be required and the general order of arguments may change, such that this metric will just function as an single entrypoint to calling the three specialized versions.
  warnings.warn(*args, **kwargs)
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
FeatureExtractor is deprecated. Use TimmFeatureExtractor instead. Both FeatureExtractor and TimmFeatureE

Model type: <class 'anomalib.models.padim.lightning_model.PadimLightning'>


## Training

Users can use each framework's training provided by OTX. (Engine)

- The engine requires the necessary models and DataLoaders for each framework.

In [6]:
from otx.v2.adapters.torch.lightning.anomalib import Engine
work_path = "/tmp/OTX-API-test"
engine = Engine(work_dir=work_path)

train_results = engine.train(
    model=model,
    train_dataloader=train_dataloader,
    max_epochs=5,
)

print(train_results["checkpoint"])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
Missing logger folder: /tmp/OTX-API-test/20231016_151349_train
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/core/optimizer.py:183: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | image_threshold       | AnomalyScoreThreshold    | 0     
1 | pixel_threshold       | AnomalyScoreThreshold    | 0     
2 | model                 | PadimModel               | 2.8 M 
3 | normalization_metrics | MinMax                   | 0     
4 | image_metrics         | AnomalibMetricCollection | 0     
5 | pixel_metrics         | 

Epoch 0: 100%|██████████| 1/1 [00:02<00:00,  2.27s/it, loss=nan, v_num=0]

/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:138: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Epoch 4: 100%|██████████| 1/1 [00:01<00:00,  1.50s/it, loss=nan, v_num=0]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it, loss=nan, v_num=0]
/tmp/OTX-API-test/20231016_151349_train/models/weights.pth


In [7]:
train_results = engine.train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    val_interval=1,
    max_epochs=2,
)

print(train_results["checkpoint"])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | image_threshold       | AnomalyScoreThreshold    | 0     
1 | pixel_threshold       | AnomalyScoreThreshold    | 0     
2 | model                 | PadimModel               | 2.8 M 
3 | normalization_metrics | MinMax                   | 0     
4 | image_metrics         | AnomalibMetricCollection | 0     
5 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1: 100%|██████████| 2/2 [00:09<00:00,  4.92s/it, loss=nan, v_num=1]  

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 2/2 [00:10<00:00,  5.02s/it, loss=nan, v_num=1]
/tmp/OTX-API-test/20231016_151349_train/models/weights.pth


In [8]:
# Validation & Testing
val_score = engine.validate(val_dataloader=val_dataloader)
print(f"Val Metric: {val_score}")

test_score = engine.test(test_dataloader=test_dataloader)
print(f"Test Metric: {test_score}")

Restoring states from the checkpoint path at /tmp/OTX-API-test/20231016_151349_train/models/weights.pth
Loaded model weights from checkpoint at /tmp/OTX-API-test/20231016_151349_train/models/weights.pth


Validation DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]

/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
Missing logger folder: /tmp/OTX-API-test/20231016_151349_test
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/


Val Metric: [{}]
Testing DataLoader 0: 100%|██████████| 23/23 [00:04<00:00,  5.61it/s]
Test Metric: [{}]


## Prediction

In [9]:
pred_results = engine.predict(
    img="../../../../tests/assets/anomaly/hazelnut/test/colour/01.jpg",
    checkpoint=train_results["checkpoint"]
)
print(pred_results)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
Missing logger folder: /tmp/OTX-API-test/20231016_151349_predict
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.99it/s]
[{'image': tensor([[[[ 1.7523,  1.8037,  1.7352,  ...,  1.6153,  1.6495,  1.6495],
          [ 1.7694,  1.7865,  1.7009,  ...,  1.6838,  1.6667,  1.6667],
          [ 1.7352,  1.7180,  1.7523,  ...,  1.6838,  1.7009,  1.6838],
          ...,
          [ 1.7009,  1.7694,  1.7352,  ...,  1.6838,  1.5982,  1.6153],
          [ 1.8208,  1.8037,  1.7865,  ...,  1.6495,  1.6495,  1.6324],
          [ 1.7865,  1.8037,  1.7694,  ...,  1.6324,  1.6324,  1.6324]],

         [[ 1.6933,  1.7283,  1.6758,  ...,  1.5532,  1.5882,  1.5882],
          [ 1.7108,  1.7108,  1.6408,  ...,  1.6232,  1.6057,  1.6057],
          [ 1.6583,  1.6408,  1.6758,  ...,  1.6232,  1.6408,  1.6232],
          ...,
          [ 1.6232,  1.6933,  1.6758,  ...,  1.6232,  1.5357,  1.5532],
          [ 1.7458,  1.7283,  1.7283,  ...,  1.5882,  1.5882,  1.5707],
          [ 1.7108,  1.7458,  1.7283,  ...,  1.5707,  1.5707,  1.5707]],

         [[-0.3753, 

## Exporting

In [10]:
export_result = engine.export(
    checkpoint=train_results["checkpoint"]
)

print(export_result)

/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:306: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/torch/onnx/utils.py:689: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/torch/onnx/utils.py:1186: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/pa

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /tmp/OTX-API-test/20231016_151349_export/openvino/openvino.xml
[ SUCCESS ] BIN file: /tmp/OTX-API-test/20231016_151349_export/openvino/openvino.bin
{'outputs': {'onnx': '/tmp/OTX-API-test/20231016_151349_export/onnx/onnx_model.onnx', 'bin': '/tmp/OTX-API-test/20231016_151349_export/openvino/openvino.bin', 'xml':

## OTX AutoRunner (Automation Training API)
OTX provides a more convenient API called AutoRunner.

- It's more convenient for users to use Engine, which provides auto-configuration and the features provided by OTX without having to choose a framework.
- Prepare Dataset & DataLoader + Prepare Model + OTX Recipes + Training + ETC.
- This will make all of the above steps happen automatically. (Auto: Model Selection & build, Dataset Configuration, Training, etc..)

In [12]:
from otx.v2.api.core import AutoRunner

output_dir = "/tmp/OTX-API-test"
data_roots = "../../../../tests/assets/anomaly/hazelnut/train"
default_config_path = "../configs/anomaly_classification/otx_anomalib_default.yaml"

engine = AutoRunner(
    task="anomaly_classification",  # TODO: Need to add Automation-task detection for Anomaly
    work_dir=output_dir,
    train_data_roots=data_roots,
    config=default_config_path,
)

engine.train(batch_size=2, max_epochs=5)

/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: DeprecationWarning: From v0.10 an `'Binary*'`, `'Multiclass*', `'Multilabel*'` version now exist of each classification metric. Moving forward we recommend using these versions. This base metric will still work as it did prior to v0.10 until v0.11. From v0.11 the `task` argument introduced in this metric will be required and the general order of arguments may change, such that this metric will just function as an single entrypoint to calling the three specialized versions.
  warnings.warn(*args, **kwargs)
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
FeatureExtractor is deprecated. Use TimmFeatureExtractor instead. Both FeatureExtractor and TimmFeatureE

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_predict_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
/home/harimkan/workspace/otx/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
Missing logger folder: /tmp/OTX-API-test/20231016_151522_train
Cannot perform pixel-level evaluation when task type is classification. Ignoring 

Epoch 4: 100%|██████████| 14/14 [00:01<00:00, 10.81it/s, loss=nan, v_num=0]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 14/14 [00:01<00:00,  8.63it/s, loss=nan, v_num=0]


{'model': PadimLightning(
   (image_threshold): AnomalyScoreThreshold()
   (pixel_threshold): AnomalyScoreThreshold()
   (model): PadimModel(
     (feature_extractor): FeatureExtractor(
       (feature_extractor): FeatureListNet(
         (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act1): ReLU(inplace=True)
         (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
         (layer1): Sequential(
           (0): BasicBlock(
             (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
             (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (drop_block): Identity()
             (act1): ReLU(inplace=True)
             (aa): Identity()
             (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(